In [1]:
import requests 
from bs4 import BeautifulSoup
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException,StaleElementReferenceException
import re

In [2]:
URL = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
driver = webdriver.Chrome(executable_path='/Applications/chromedriver')
driver.get(URL)
wait = WebDriverWait(driver, 10)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [3]:
col=['title','year','genre','rating','director','actors','votes','url']
imdb_df = pd.DataFrame(columns=col)   
genre_click = soup.find_all(class_='subnav_item_main')
for item in genre_click:                              #open a new page for each genre
    newlink = 'https://www.imdb.com'+item.a['href']
    driver.get(newlink)
    wait = WebDriverWait(driver, 10)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    page_count = 0       
    time.sleep(20)
    while True:                                      #scrap the first 5 pages of each genre movie
        movie = soup.find_all(class_='lister-item mode-advanced')
        for m in movie:
            try:                                     #title
                title_text = m.find(class_="lister-item-header").text
                title_text = title_text.replace('\n','')
                title = re.search(r'\d+\.(.*)\(\d{4}\)',title_text).group(1)
                year = re.search(r'\((\d{4})\)$',title_text).group(1)
            except:
                title = np.nan
                year = np.nan
            try:                                     #genre
                genre = m.find(class_="genre").text.strip()
            except:
                genre = np.nan
            try:                                     #rating
                rating = m.find(class_="inline-block ratings-imdb-rating").text.strip()
            except:
                rating = np.nan
            try:                                     #director,actor
                all_people = m.find(class_="sort-num_votes-visible").find_previous_sibling("p")
                all_people = all_people.text.replace('\n','').strip()
                director = re.search(r"Directors?\:(.*)\|",all_people).group(1)
                actors = re.search(r"Stars?\:(.*)",all_people).group(1)
            except:
                all_people = np.nan
            try:                                     #votes number
                votes = m.find(class_="sort-num_votes-visible").find(attrs={"name": "nv"}).text
                votes = int(votes.replace(',',''))                    
            except:
                votes = np.nan
            try:
                movie_link = m.find(class_="lister-item-image float-left").a['href']
                movie_link = 'https://www.imdb.com' + movie_link
            except:
                movie_link = np.nan
                
            print(title,year,genre,rating,director,actors,votes,movie_link)
            print('')
            imdb_df = imdb_df.append({'title':title,'year':year,'genre':genre,'rating':rating,
                                      'director':director,'actors':actors,'votes':votes,'url':movie_link},ignore_index=True)
        next_page = soup.find(class_="lister-page-next next-page")
        if next_page and page_count <= 5:
            page_count += 1
            next_page_link = 'https://www.imdb.com'+next_page['href']
            driver.get(next_page_link)
            wait = WebDriverWait(driver, 10)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            time.sleep(20)
        else:
            break
imdb_df.to_csv('imdb_new.csv')

The Dark Knight 2008 Action, Crime, Drama 9.0 Christopher Nolan Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine 2222222 https://www.imdb.com/title/tt0468569/?ref_=adv_li_i

The Mountain II 2016 Action, Drama, War 8.9 Alper Caglar Caglar Ertugrul, Ufuk Bayraktar, Ahu Türkpençe, Murat Serezli 103623 https://www.imdb.com/title/tt5813916/?ref_=adv_li_i

Inception 2010 Action, Adventure, Sci-Fi 8.8 Christopher Nolan Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Ken Watanabe 1978620 https://www.imdb.com/title/tt1375666/?ref_=adv_li_i

The Lord of the Rings: The Fellowship of the Ring 2001 Action, Adventure, Drama 8.8 Peter Jackson Elijah Wood, Ian McKellen, Orlando Bloom, Sean Bean 1606922 https://www.imdb.com/title/tt0120737/?ref_=adv_li_i

The Matrix 1999 Action, Sci-Fi 8.7 Lana Wachowski, Lilly Wachowski Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving 1617768 https://www.imdb.com/title/tt0133093/?ref_=adv_li_i

Star Wars: Episode V - The Empire Strikes B

Jurassic Park 1993 Action, Adventure, Sci-Fi 8.1 Steven Spielberg Sam Neill, Laura Dern, Jeff Goldblum, Richard Attenborough 835480 https://www.imdb.com/title/tt0107290/?ref_=adv_li_i

Blade Runner 1982 Action, Sci-Fi, Thriller 8.1 Ridley Scott Harrison Ford, Rutger Hauer, Sean Young, Edward James Olmos 672275 https://www.imdb.com/title/tt0083658/?ref_=adv_li_i

Sanjuro 1962 Action, Comedy, Crime 8.1 Akira Kurosawa Toshirô Mifune, Tatsuya Nakadai, Keiju Kobayashi, Yûnosuke Itô 30828 https://www.imdb.com/title/tt0056443/?ref_=adv_li_i

White Heat 1949 Action, Crime, Drama 8.1 Raoul Walsh James Cagney, Virginia Mayo, Edmond O'Brien, Margaret Wycherly 28632 https://www.imdb.com/title/tt0042041/?ref_=adv_li_i

The General 1926 Action, Adventure, Comedy 8.1 Clyde Bruckman, Buster Keaton Buster Keaton, Marion Mack, Glen Cavender, Jim Farley 77818 https://www.imdb.com/title/tt0017925/?ref_=adv_li_i

Bajrangi Bhaijaan 2015 Action, Comedy, Drama 8.0 Kabir Khan Salman Khan, Harshaali Malhotra, N

Big Hero 6 2014 Animation, Action, Adventure 7.8 Don Hall, Chris Williams Ryan Potter, Scott Adsit, Jamie Chung, T.J. Miller 397647 https://www.imdb.com/title/tt2245084/?ref_=adv_li_i

How to Train Your Dragon 2 2014 Animation, Action, Adventure 7.8 Dean DeBlois Jay Baruchel, Cate Blanchett, Gerard Butler, Craig Ferguson 296251 https://www.imdb.com/title/tt1646971/?ref_=adv_li_i

I Saw the Devil 2010 Action, Crime, Drama 7.8 Jee-woon Kim Byung-hun Lee, Min-sik Choi, Gook-hwan Jeon, Joon-hyuk Lee 104422 https://www.imdb.com/title/tt1588170/?ref_=adv_li_i

The Man from Nowhere 2010 Action, Drama, Thriller 7.8 Jeong-beom Lee Won Bin, Sae-ron Kim, Tae-hoon Kim, Hee-won Kim 60126 https://www.imdb.com/title/tt1527788/?ref_=adv_li_i

Taken(I)  2008 Action, Thriller 7.8 Pierre Morel Liam Neeson, Maggie Grace, Famke Janssen, Leland Orser 555499 https://www.imdb.com/title/tt0936501/?ref_=adv_li_i

Avatar 2009 Action, Adventure, Fantasy 7.8 James Cameron Sam Worthington, Zoe Saldana, Sigourney We

Goldfinger 1964 Action, Adventure, Thriller 7.7 Guy Hamilton Sean Connery, Gert Fröbe, Honor Blackman, Shirley Eaton 167597 https://www.imdb.com/title/tt0058150/?ref_=adv_li_i

The Magnificent Seven 1960 Action, Adventure, Western 7.7 John Sturges Yul Brynner, Steve McQueen, Charles Bronson, Eli Wallach 85199 https://www.imdb.com/title/tt0054047/?ref_=adv_li_i

Hotel Mumbai 2018 Action, Drama, History 7.6 Anthony Maras Dev Patel, Armie Hammer, Nazanin Boniadi, Anupam Kher 47463 https://www.imdb.com/title/tt5461944/?ref_=adv_li_i

Guardians of the Galaxy: Vol. 2 2017 Action, Adventure, Comedy 7.6 James Gunn Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel 543511 https://www.imdb.com/title/tt3896198/?ref_=adv_li_i

Baby Driver 2017 Action, Crime, Drama 7.6 Edgar Wright Ansel Elgort, Jon Bernthal, Jon Hamm, Eiza González 414767 https://www.imdb.com/title/tt3890160/?ref_=adv_li_i

Only the Brave 2017 Action, Biography, Drama 7.6 Joseph Kosinski Josh Brolin, Miles Teller, Jeff Bridges, J

Summer Wars 2009 Animation, Action, Adventure 7.5 Mamoru Hosoda Ryûnosuke Kamiki, Nanami Sakuraba, Mitsuki Tanimura, Takahiro Yokokawa 25096 https://www.imdb.com/title/tt1474276/?ref_=adv_li_i

Ip Man 2 2010 Action, Biography, Drama 7.5 Wilson Yip Donnie Yen, Xiaoming Huang, Sammo Kam-Bo Hung, Lynn Xiong 94332 https://www.imdb.com/title/tt1386932/?ref_=adv_li_i

Mesrine Part 1: Killer Instinct 2008 Action, Biography, Crime 7.5 Jean-François Richet Vincent Cassel, Cécile de France, Gérard Depardieu, Gilles Lellouche 31445 https://www.imdb.com/title/tt1259014/?ref_=adv_li_i

Doctor Strange 2016 Action, Adventure, Fantasy 7.5 Scott Derrickson Benedict Cumberbatch, Chiwetel Ejiofor, Rachel McAdams, Benedict Wong 572270 https://www.imdb.com/title/tt1211837/?ref_=adv_li_i

Lone Survivor 2013 Action, Biography, Drama 7.5 Peter Berg Mark Wahlberg, Taylor Kitsch, Emile Hirsch, Ben Foster 259770 https://www.imdb.com/title/tt1091191/?ref_=adv_li_i

Source Code 2011 Action, Drama, Mystery 7.5 Dunc

...

The African Queen 1951 Adventure, Drama, Romance 7.7 John Huston Humphrey Bogart, Katharine Hepburn, Robert Morley, Peter Bull 69655 https://www.imdb.com/title/tt0043265/?ref_=adv_li_i

Fury 2014 Action, Drama, War 7.6 David Ayer Brad Pitt, Shia LaBeouf, Logan Lerman, Michael Peña 411405 https://www.imdb.com/title/tt2713180/?ref_=adv_li_i

Les Misérables 2012 Drama, History, Musical 7.6 Tom Hooper Hugh Jackman, Russell Crowe, Anne Hathaway, Amanda Seyfried 300587 https://www.imdb.com/title/tt1707386/?ref_=adv_li_i

The Flowers of War 2011 Drama, History, Romance 7.6 Yimou Zhang Christian Bale, Ni Ni, Xinyi Zhang, Tianyuan Huang 48300 https://www.imdb.com/title/tt1410063/?ref_=adv_li_i

The Hurt Locker 2008 Drama, Thriller, War 7.6 Kathryn Bigelow Jeremy Renner, Anthony Mackie, Brian Geraghty, Guy Pearce 411558 https://www.imdb.com/title/tt0887912/?ref_=adv_li_i

The Book Thief 2013 Drama, War 7.6 Brian Percival Sophie Nélisse, Geoffrey Rush, Emily Watson, Ben Schnetzer 124506 https://w

The Siege of Jadotville 2016 Action, Drama, History 7.2 Richie Smyth Richard Lukunku, Danny Sapani, Andrew Stock, Mark Strong 29127 https://www.imdb.com/title/tt3922798/?ref_=adv_li_i

Bajirao Mastani 2015 Action, Drama, History 7.2 Sanjay Leela Bhansali Ranveer Singh, Deepika Padukone, Priyanka Chopra, Tanvi Azmi 30372 https://www.imdb.com/title/tt3735246/?ref_=adv_li_i

'71 2014 Action, Crime, Drama 7.2 Yann Demange Jack O'Connell, Sam Reid, Sean Harris, Paul Popplewell 49084 https://www.imdb.com/title/tt2614684/?ref_=adv_li_i

Unbroken(I)  2014 Biography, Drama, Sport 7.2 Angelina Jolie Jack O'Connell, Miyavi, Domhnall Gleeson, Garrett Hedlund 141144 https://www.imdb.com/title/tt1809398/?ref_=adv_li_i

War Horse 2011 Action, Adventure, Drama 7.2 Steven Spielberg Jeremy Irvine, Emily Watson, David Thewlis, Benedict Cumberbatch 143269 https://www.imdb.com/title/tt1568911/?ref_=adv_li_i

Defiance(I)  2008 Action, Drama, History 7.2 Edward Zwick Daniel Craig, Liev Schreiber, Jamie Bell,

Conquest 1453 2012 Action, Drama, History 6.7 Faruk Aksoy Devrim Evin, Ibrahim Celikkol, Dilek Serbest, Cengiz Coskun 55982 https://www.imdb.com/title/tt1783232/?ref_=adv_li_i

Seraphim Falls 2006 Action, Drama, Thriller 6.7 David Von Ancken Pierce Brosnan, Liam Neeson, Anjelica Huston, Michael Wincott 28842 https://www.imdb.com/title/tt0479537/?ref_=adv_li_i

K-19: The Widowmaker 2002 Drama, History, Thriller 6.7 Kathryn Bigelow Harrison Ford, Sam Spruell, Peter Stebbings, Christian Camargo 58206 https://www.imdb.com/title/tt0267626/?ref_=adv_li_i

Escape to Victory 1981 Drama, Sport, War 6.7 John Huston Michael Caine, Sylvester Stallone, Pelé, Bobby Moore 26757 https://www.imdb.com/title/tt0083284/?ref_=adv_li_i

Operation Finale 2018 Biography, Drama, History 6.6 Chris Weitz Oscar Isaac, Ben Kingsley, Mélanie Laurent, Lior Raz 28776 https://www.imdb.com/title/tt5208252/?ref_=adv_li_i

Overlord 2018 Action, Adventure, Horror 6.6 Julius Avery Jovan Adepo, Wyatt Russell, Mathilde Olliv

1941 1979 Action, Comedy, War 5.8 Steven Spielberg John Belushi, Dan Aykroyd, Treat Williams, Nancy Allen 30587 https://www.imdb.com/title/tt0078723/?ref_=adv_li_i

Timeline 2003 Action, Adventure, Sci-Fi 5.7 Richard Donner Paul Walker, Gerard Butler, Billy Connolly, Frances O'Connor 59292 https://www.imdb.com/title/tt0300556/?ref_=adv_li_i

Alexander 2004 Action, Biography, Drama 5.6 Oliver Stone Colin Farrell, Anthony Hopkins, Rosario Dawson, Angelina Jolie 159740 https://www.imdb.com/title/tt0346491/?ref_=adv_li_i

The Last Legion 2007 Action, Adventure, War 5.4 Doug Lefler Colin Firth, Ben Kingsley, Aishwarya Rai Bachchan, Peter Mullan 33684 https://www.imdb.com/title/tt0462396/?ref_=adv_li_i

Pathfinder 2007 Action, Adventure, Drama 5.4 Marcus Nispel Karl Urban, Clancy Brown, Moon Bloodgood, Russell Means 41921 https://www.imdb.com/title/tt0446013/?ref_=adv_li_i

In the Land of Blood and Honey 2011 Drama, Romance, War 4.5 Angelina Jolie Zana Marjanovic, Goran Kostic, Rade Serbedzi

Brimstone 2016 Drama, Mystery, Thriller 7.1 Martin Koolhoven Guy Pearce, Dakota Fanning, Emilia Jones, Carice van Houten 34486 https://www.imdb.com/title/tt1895315/?ref_=adv_li_i

Spirit: Stallion of the Cimarron 2002 Animation, Adventure, Drama 7.1 Kelly Asbury, Lorna Cook Matt Damon, James Cromwell, Daniel Studi, Chopper Bernet 62750 https://www.imdb.com/title/tt0166813/?ref_=adv_li_i

The Sisters Brothers 2018 Crime, Drama, Western 7.0 Jacques Audiard John C. Reilly, Joaquin Phoenix, Jake Gyllenhaal, Riz Ahmed 48540 https://www.imdb.com/title/tt4971344/?ref_=adv_li_i

Bubba Ho-Tep 2002 Comedy, Fantasy, Horror 7.0 Don Coscarelli Bruce Campbell, Ossie Davis, Bob Ivy, Ella Joyce 44754 https://www.imdb.com/title/tt0281686/?ref_=adv_li_i

Maverick 1994 Action, Adventure, Comedy 7.0 Richard Donner Mel Gibson, Jodie Foster, James Garner, Graham Greene 99172 https://www.imdb.com/title/tt0110478/?ref_=adv_li_i

Westworld 1973 Action, Sci-Fi, Thriller 7.0 Michael Crichton Yul Brynner, Richard